In [3]:
import numpy as np
import matplotlib.pyplot as plt

# Requirement 1
## Stochastic environment

### Price : uniform distribution

### Demand curve

In [19]:

price = 10
conversion_probability = lambda p: 1-p/20

sold = np.random.binomial(1, conversion_probability(price))

print(f'Probability of purchase at price {price}: {conversion_probability(price)}')

# will the customer buy my product at this price?
print(sold)
print(f'Sold? {bool(sold)}')

Probability of purchase at price 15: 0.25
2
Sold? True


# Pricing Environment

In [1]:
class PricingEnvironment:
    def __init__(self, conversion_probability, cost):
        self.conversion_probability = conversion_probability
        self.cost = cost

    def round(self, p_t, n_t):
        d_t = np.random.binomial(n_t, self.conversion_probability(p_t))
        r_t = (p_t - self.cost)*d_t
        return d_t, r_t
    

In [2]:
env = PricingEnvironment(conversion_probability=conversion_probability, cost=5)
env.round(10, 100)

NameError: name 'conversion_probability' is not defined